# Mass Regressin with Decision Trees
In this notebook we will explore how to apply boosted decision trees (BDT) to the 2-D mass plane.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uproot as ur
import awkward as ak

print('Numpy version: {}'.format(np.__version__))
print('Uproot version: {}'.format(ur.__version__))
print('Awkward version: {}'.format(ak.__version__))

Numpy version: 1.23.1
Uproot version: 4.3.3
Awkward version: 1.8.0


## Uproot Files

In [2]:
rootfile_prefix = '/fast_scratch_1/atlas_images/XhhNTuple/'
rfile = 'user.zhenw.29137978._000001.MiniNTuple.root'

In [3]:
uprooted = ur.open(rootfile_prefix+rfile)
uprooted.keys()

['XhhMiniNtuple;1',
 'cutflow_XhhMiniNtuple;1',
 'cutflow_weighted_XhhMiniNtuple;1',
 'MetaData_EventCount_XhhMiniNtuple;1']

In [4]:
MNTuple = uprooted['XhhMiniNtuple']
# events.arrays(["px1", "py1", "pz1"])
MNTuple.show(name_width=32,
            interpretation_width=30)

name                             | typename                 | interpretation                
---------------------------------+--------------------------+-------------------------------
runNumber                        | int32_t                  | AsDtype('>i4')
eventNumber                      | int64_t                  | AsDtype('>i8')
lumiBlock                        | int32_t                  | AsDtype('>i4')
coreFlags                        | uint32_t                 | AsDtype('>u4')
bcid                             | int32_t                  | AsDtype('>i4')
mcEventNumber                    | int32_t                  | AsDtype('>i4')
mcChannelNumber                  | int32_t                  | AsDtype('>i4')
mcEventWeight                    | float                    | AsDtype('>f4')
NPV                              | int32_t                  | AsDtype('>i4')
actualInteractionsPerCrossing    | float                    | AsDtype('>f4')
averageInteractionsPerCrossing   | float    

In [5]:
import os
import sys
cwd = os.getcwd()
path_head, path_tail = os.path.split(cwd)
sys.path.append(path_head+'/utils')
from ml_utils import dict_from_tree, DeltaR
from time import perf_counter as cput

In [6]:
branches = ['boosted_nGoodJets', "nboostedJets", "boostedJets_m",
            "boostedJets_pt", "boostedJets_phi", "boostedJets_eta",
           "truth_mHH", 'truthjet_antikt10_pt', 'truthjet_antikt10_eta',
           'truthjet_antikt10_phi', 'truthjet_antikt10_m']
np_branches = ['eventNumber']

In [7]:
t0 = cput()
hh4b_dict = dict_from_tree(MNTuple, branches, np_branches)
t1 = cput()
method_1_time = t1 - t0
print('Time to load arrays: {:8.4f} (s)'.format(method_1_time))

nEvents = len(hh4b_dict['eventNumber'])
print('{} Events'.format(nEvents))

Time to load arrays:   1.6890 (s)
339978 Events


In [8]:
gt_twoJets = np.zeros(nEvents, dtype=bool)
t0 = cput()
for i in range(nEvents):
    boostedJets_pt = ak.to_numpy(hh4b_dict['boostedJets_pt'][i])
    
    if len(boostedJets_pt) >=2:
        gt_twoJets[i] = True

n2jets = np.count_nonzero(gt_twoJets)
print(n2jets)
t1 = cput()
print(t1 - t0)

95892
29.62432041298598


In [9]:
list_indices = np.arange(nEvents)
# evt_idx = np.full(shape=(n2jets,), fill_value=-1)
evt_idx = np.ndarray.copy(list_indices[gt_twoJets])

for jk in range(20):
    evt = evt_idx[jk]
    nboosted = len(ak.to_numpy(hh4b_dict['boostedJets_pt'][evt]))
    print('Event with matched jets: {}'.format(evt_idx[jk]))
    print('Number of boosted jets:   {}'.format(nboosted))

Event with matched jets: 1
Number of boosted jets:   2
Event with matched jets: 3
Number of boosted jets:   2
Event with matched jets: 9
Number of boosted jets:   2
Event with matched jets: 14
Number of boosted jets:   2
Event with matched jets: 20
Number of boosted jets:   3
Event with matched jets: 27
Number of boosted jets:   2
Event with matched jets: 29
Number of boosted jets:   2
Event with matched jets: 33
Number of boosted jets:   2
Event with matched jets: 34
Number of boosted jets:   2
Event with matched jets: 36
Number of boosted jets:   2
Event with matched jets: 50
Number of boosted jets:   2
Event with matched jets: 51
Number of boosted jets:   2
Event with matched jets: 54
Number of boosted jets:   2
Event with matched jets: 55
Number of boosted jets:   2
Event with matched jets: 56
Number of boosted jets:   2
Event with matched jets: 58
Number of boosted jets:   2
Event with matched jets: 62
Number of boosted jets:   2
Event with matched jets: 67
Number of boosted jets:

In [10]:
t0 = cput()
matched_jets = []
for i, evt in enumerate(evt_idx):
    
    nTruthJets = ak.to_numpy(hh4b_dict['truthjet_antikt10_m'][evt]).shape[0]
    truthJet_coords = np.empty((nTruthJets, 2))
    # this can be vectorized easily (too tired)
    for j in range(nTruthJets):
        truthJet_coords[j,0] = hh4b_dict['truthjet_antikt10_eta'][evt][j]
        truthJet_coords[j,1] = hh4b_dict['truthjet_antikt10_phi'][evt][j]
    
    BoostedJet0_eta = hh4b_dict['boostedJets_eta'][evt][0]
    BoostedJet0_phi = hh4b_dict['boostedJets_phi'][evt][0]
    BoostedJet0_coords = np.array([BoostedJet0_eta, BoostedJet0_phi])
    LeadingJet_DR_arr = DeltaR(truthJet_coords, BoostedJet0_coords)
    # print(LeadingJet_DR_arr)
    LJ_DR = np.min(LeadingJet_DR_arr)
    LJ_DR_idx = np.argmin(LeadingJet_DR_arr)
    
    BoostedJet1_eta = hh4b_dict['boostedJets_eta'][evt][1]
    BoostedJet1_phi = hh4b_dict['boostedJets_phi'][evt][1]
    BoostedJet1_coords = np.array([BoostedJet1_eta, BoostedJet1_phi])
    subLeadingJet_DR_arr = DeltaR(truthJet_coords, BoostedJet1_coords)
    # print(subLeadingJet_DR_arr)
    SLJ_DR = np.min(subLeadingJet_DR_arr)
    SLJ_DR_idx = np.argmin(subLeadingJet_DR_arr)
    
    if SLJ_DR_idx != LJ_DR_idx:
        if LJ_DR < .1 and SLJ_DR < .1:
            matched_jets.append([evt, LJ_DR_idx, SLJ_DR_idx])
    
    # print();print()

t1 = cput()
print('Time to complete jet matching: {:6.3f} (m)'.format((t1 - t0)/60))
print();print()
matched_jets = np.array(matched_jets)

print(matched_jets.shape)

Time to complete jet matching:  1.643 (m)


(94859, 3)


In [11]:
X = np.empty((matched_jets.shape[0],8))
Y = np.empty((matched_jets.shape[0],2))

for i in range(matched_jets.shape[0]):
    arr_slc = matched_jets[i]
    evt = arr_slc[0]
    ld_idx = arr_slc[1]
    sld_idx = arr_slc[2]

    nboosted = len(ak.to_numpy(hh4b_dict['boostedJets_pt'][evt]))
    
    # Leading
    X[i,0] = hh4b_dict['boostedJets_m'][evt][0]
    X[i,1] = hh4b_dict['boostedJets_pt'][evt][0]
    X[i,2] = hh4b_dict['boostedJets_eta'][evt][0]
    X[i,3] = hh4b_dict['boostedJets_phi'][evt][0]
    
    # Sub-Leading
    X[i,4] = hh4b_dict['boostedJets_m'][evt][1]
    X[i,5] = hh4b_dict['boostedJets_pt'][evt][1]
    X[i,6] = hh4b_dict['boostedJets_eta'][evt][1]
    X[i,7] = hh4b_dict['boostedJets_phi'][evt][1]
    
    # Truth Jet
    Y[i,0] = hh4b_dict['truthjet_antikt10_m'][evt][0]
    Y[i,1] = hh4b_dict['truthjet_antikt10_m'][evt][1]

## Build Decision Trees (?)

In [12]:
import xgboost as xgb